In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# !pip install pandas plotly dash

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# import plotly.express as px
# from dash import Dash, dcc, html, Input, Output


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Read the CSV file into a DataFrame
df = pd.read_csv("/kaggle/input/dallas-public-safety/Police_Incidents.csv")

# Display the DataFrame
# print(df["Service Number ID"])


In [ ]:
# # Group the data and calculate incident counts
# grouped = df.groupby(['Sector', 'Division', 'Day1 of the Week', 'Time1 of Occurrence']).size().reset_index(name='IncidentCount')

# # Define a scoring function (you can customize this based on your preference)
# def calculate_safety_score(incident_count):
#     return 100 / (incident_count + 1)  # Example scoring function (higher count -> lower score)

# # Calculate safety scores based on incident counts
# grouped['SafetyScore'] = grouped['IncidentCount'].apply(calculate_safety_score)

# # Merge safety scores back into the original dataset
# df = pd.merge(df, grouped[['Sector', 'Division', 'Day1 of the Week', 'Time1 of Occurrence', 'SafetyScore']], on=['Sector', 'Division', 'Day1 of the Week', 'Time1 of Occurrence'], how='left')

# # Display the updated DataFrame with the 'SafetyScore' column
# print(df['SafetyScore'])

In [ ]:


# # Create the initial division-based graph
# division_fig = px.bar(df, x='Division', title='Division-based Incident Counts')
# division_fig.update_xaxes(categoryorder='total descending')

# # Create callback functions for interactive filtering
# def update_sector_graph(selected_division):
#     filtered_df = df[df['Division'] == selected_division]
#     sector_fig = px.bar(filtered_df, x='Sector', title=f'Sectors in Division {selected_division}')
#     sector_fig.update_xaxes(categoryorder='total descending')
#     return sector_fig

# def update_day_graph(selected_sector):
#     filtered_df = df[df['Sector'] == selected_sector]
#     day_fig = px.bar(filtered_df, x='Day1 of the Week', title=f'Days for Sector {selected_sector}')
#     day_fig.update_xaxes(categoryorder=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])
#     return day_fig

# def update_time_graph(selected_day):
#     filtered_df = df[df['Day1 of the Week'] == selected_day]
#     time_fig = px.bar(filtered_df, x='Time1 of Occurrence', title=f'Time on {selected_day}')
#     time_fig.update_xaxes(categoryorder='total ascending')
#     return time_fig

# # Create the interactive app

# app = Dash(__name__)

# app.layout = html.Div([
#     dcc.Graph(id='division-graph', figure=division_fig),
#     dcc.Graph(id='sector-graph'),
#     dcc.Graph(id='day-graph'),
#     dcc.Graph(id='time-graph')
# ])

# @app.callback(
#     Output('sector-graph', 'figure'),
#     Input('division-graph', 'clickData')
# )
# def update_sector_graph_callback(selected_data):
#     if selected_data is None:
#         return px.bar()
#     selected_division = selected_data['points'][0]['x']
#     return update_sector_graph(selected_division)

# @app.callback(
#     Output('day-graph', 'figure'),
#     Input('sector-graph', 'clickData')
# )
# def update_day_graph_callback(selected_data):
#     if selected_data is None:
#         return px.bar()
#     selected_sector = selected_data['points'][0]['x']
#     return update_day_graph(selected_sector)

# @app.callback(
#     Output('time-graph', 'figure'),
#     Input('day-graph', 'clickData')
# )
# def update_time_graph_callback(selected_data):
#     if selected_data is None:
#         return px.bar()
#     selected_day = selected_data['points'][0]['x']
#     return update_time_graph(selected_day)

# if __name__ == '__main__':
#     app.run_server(debug=True, mode='inline')


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact, widgets
# Take the first 1000 entries as a subset
subset_df = df.head(1000)

# Group the subset data and calculate incident counts
grouped = subset_df.groupby(['Division', 'Sector', 'Day1 of the Week', 'Time1 of Occurrence']).size().reset_index(name='IncidentCount')

# Create a function to update and display the bar chart
def plot_data(division=None, sector=None, day=None):
    filtered_df = grouped.copy()
    
    if division:
        filtered_df = filtered_df[filtered_df['Division'] == division]
    
    if sector:
        filtered_df = filtered_df[filtered_df['Sector'] == sector]
    
    if day:
        filtered_df = filtered_df[filtered_df['Day1 of the Week'] == day]
    
    plt.figure(figsize=(10, 6))
    plt.bar(filtered_df['Time1 of Occurrence'], filtered_df['IncidentCount'])
    plt.xlabel('Time of Occurrence')
    plt.ylabel('Incident Count')
    plt.title('Incident Counts')
    plt.xticks(rotation=45)
    plt.show()

# Create interactive widgets for filtering
division_widget = widgets.Dropdown(
    options=['All'] + grouped['Division'].unique().tolist(),
    value='All',
    description='Division:'
)

sector_widget = widgets.Dropdown(
    options=['All'] + grouped['Sector'].unique().tolist(),
    value='All',
    description='Sector:'
)

day_widget = widgets.Dropdown(
    options=['All'] + grouped['Day1 of the Week'].unique().tolist(),
    value='All',
    description='Day:'
)

# Define an interactive function to update the chart based on widget selections
def update_chart(division, sector, day):
    plot_data(division if division != 'All' else None,
              sector if sector != 'All' else None,
              day if day != 'All' else None)

# Create an interactive dashboard using widgets
interact(update_chart, division=division_widget, sector=sector_widget, day=day_widget)

# Initialize the chart with all data
plot_data()